In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import csv

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [9]:
# load the training data
with open('./data/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

train_images = np.array(train_data['images'])
train_labels = np.array(train_data['labels'])

# train_images = train_images.astype(np.float32)/255.0

In [16]:
import pandas as pd
train_images

array([[[0.        , 0.        , 0.08627451, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.08627451, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.08627451, ..., 0.09411765,
         0.09803922, 0.09803922],
        ...,
        [0.03921569, 0.03921569, 0.03921569, ..., 0.18039216,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.15686275,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.1254902 ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.10980392, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.12156863, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.14117648, ..., 0.3019608 ,
         0.38039216, 0.4       ],
        ...,
        [0.7058824 , 0.6666667 , 0.62352943, ..., 0.05882353,
         0.        , 0.        ],
        [0. 

In [18]:
train_labels

array([0, 3, 3, ..., 0, 3, 0])

In [10]:
# data augumentation and transformers
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [11]:
# Data preparation for PyTorch
train_images = np.expand_dims(train_images, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Convert to tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

In [12]:
# Create data loaders
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# Define the CNN model
class RetinalDiseaseClassifier(nn.Module):
    def __init__(self):
        super(RetinalDiseaseClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Output: (32, 28, 28)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Output: (32, 14, 14)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Output: (64, 14, 14)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Output: (64, 7, 7)
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Output: (128, 7, 7)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)   # Output: (128, 3, 3)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 4)  # Output classes: 4
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

In [14]:
# Model initialization, loss function and optimizer
model = RetinalDiseaseClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# training loop
num_epochs = 20
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)
    
    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples
    
    # Validation phase
    model.eval()
    val_running_corrects = 0
    val_total_samples = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = val_inputs.to(device)
            val_labels = val_labels.to(device)
            val_outputs = model(val_inputs)
            _, val_preds = torch.max(val_outputs, 1)
            val_running_corrects += torch.sum(val_preds == val_labels.data)
            val_total_samples += val_inputs.size(0)
    
    val_acc = val_running_corrects.double() / val_total_samples
    
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
          f"Val Acc: {val_acc:.4f}")
    
    # Save the model if it has the best validation accuracy so far
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

Epoch 1/20, Train Loss: 0.6038, Train Acc: 0.7920, Val Acc: 0.4721
Epoch 2/20, Train Loss: 0.4618, Train Acc: 0.8415, Val Acc: 0.4721
Epoch 3/20, Train Loss: 0.4147, Train Acc: 0.8559, Val Acc: 0.4721
Epoch 4/20, Train Loss: 0.3787, Train Acc: 0.8676, Val Acc: 0.4721
Epoch 5/20, Train Loss: 0.3549, Train Acc: 0.8769, Val Acc: 0.3440
Epoch 6/20, Train Loss: 0.3291, Train Acc: 0.8847, Val Acc: 0.4711
Epoch 7/20, Train Loss: 0.3090, Train Acc: 0.8907, Val Acc: 0.4723
Epoch 8/20, Train Loss: 0.2870, Train Acc: 0.8980, Val Acc: 0.4721
Epoch 9/20, Train Loss: 0.2701, Train Acc: 0.9042, Val Acc: 0.0814
Epoch 10/20, Train Loss: 0.2501, Train Acc: 0.9100, Val Acc: 0.1049
Epoch 11/20, Train Loss: 0.2338, Train Acc: 0.9159, Val Acc: 0.3599
Epoch 12/20, Train Loss: 0.2176, Train Acc: 0.9214, Val Acc: 0.0796
Epoch 13/20, Train Loss: 0.2060, Train Acc: 0.9251, Val Acc: 0.1168
Epoch 14/20, Train Loss: 0.1891, Train Acc: 0.9317, Val Acc: 0.1048
Epoch 15/20, Train Loss: 0.1759, Train Acc: 0.9359, Val A

In [16]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

C:\Users\ns99a\AppData\Local\Temp\ipykernel_27512\869501355.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


<All keys matched successfully>

In [17]:
# Test
with open('./data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

test_images = np.array(test_data['images'])
test_images = test_images.astype(np.float32) / 255.0
test_images = np.expand_dims(test_images, axis=1)  # Add channel dimension

X_test_tensor = torch.tensor(test_images)

test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Make predictions on the test data
model.eval()
all_preds = []
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs[0].to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())


In [18]:
with open('submission_v2.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'Class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for idx, pred in enumerate(all_preds):
        writer.writerow({'ID': idx + 1, 'Class': int(pred)})

---

In [ ]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [1]:
# Import necessary libraries
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

# Define a custom dataset class
class RetinalDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images.astype(np.float32)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = np.expand_dims(image, axis=2)  # Add channel dimension
        if self.transform:
            image = self.transform(image)
        else:
            image = torch.tensor(image.transpose((2, 0, 1)))  # Convert to (C, H, W)
        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.long)  # Convert label to torch.LongTensor
            return image, label
        else:
            return image

# Load and preprocess the data
# Load the training data
with open('./data/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

train_images = np.array(train_data['images'])
train_labels = np.array(train_data['labels'])

# Define transformations with data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])  # Normalization for pre-trained models
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

# Create datasets
train_dataset = RetinalDataset(X_train, y_train, transform=train_transform)
val_dataset = RetinalDataset(X_val, y_val, transform=val_transform)

# Hyperparameters
model_name = 'resnet34'
learning_rate = 1e-4
batch_size = 16
num_epochs = 10  # Increase for better results

# Update data loaders with batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Load pre-trained model
if model_name == 'resnet18':
    model = models.resnet18(weights='IMAGENET1K_V1')
elif model_name == 'resnet34':
    model = models.resnet34(weights='IMAGENET1K_V1')

# Modify the first layer to accept single-channel (grayscale) images
model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the last layer to output 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Move the model to CPU (no GPU acceleration)
model = model.cpu()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
best_val_accuracy = 0.0

# Lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in train_loader:
        # Move inputs and labels to CPU
        inputs = inputs.cpu()
        labels = labels.cpu()

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc.item())

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    val_total_samples = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            # Move inputs and labels to CPU
            val_inputs = val_inputs.cpu()
            val_labels = val_labels.cpu()
            val_outputs = model(val_inputs)
            _, val_preds = torch.max(val_outputs, 1)
            val_loss = criterion(val_outputs, val_labels)

            val_running_loss += val_loss.item() * val_inputs.size(0)
            val_running_corrects += torch.sum(val_preds == val_labels.data)
            val_total_samples += val_inputs.size(0)

    val_epoch_loss = val_running_loss / val_total_samples
    val_acc = val_running_corrects.double() / val_total_samples

    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_acc.item())

    # Print metrics
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Save the model if it has the best validation accuracy so far
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

end_time = time.time()
print(f"Training complete in {(end_time - start_time)/60:.2f} minutes")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

# Plot training and validation loss over epochs
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.legend()

# Plot training and validation accuracy over epochs
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, num_epochs+1), val_accuracies, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

# Load the best model and make predictions on the test set
# Load the best saved model
if model_name == 'resnet18':
    best_model = models.resnet18(weights='IMAGENET1K_V1')
elif model_name == 'resnet34':
    best_model = models.resnet34(weights='IMAGENET1K_V1')

# Modify the first layer to accept single-channel images
best_model.conv1 = nn.Conv2d(1, best_model.conv1.out_channels, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the last layer
num_ftrs = best_model.fc.in_features
best_model.fc = nn.Linear(num_ftrs, 4)

best_model.load_state_dict(torch.load('best_model.pth'))
# Move the model to CPU
best_model = best_model.cpu()
best_model.eval()

# Load the test data
with open('./data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

test_images = np.array(test_data['images'])
test_dataset = RetinalDataset(test_images, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# Make predictions on the test data
all_preds = []
with torch.no_grad():
    for inputs in test_loader:
        # Move inputs to CPU
        inputs = inputs.cpu()
        outputs = best_model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

# Write the predictions to a CSV file
import csv

with open('submission_v3.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'Class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for idx, pred in enumerate(all_preds):
        writer.writerow({'ID': idx + 1, 'Class': int(pred)})

print("Submission file 'submission_best_model.csv' has been created.")

KeyboardInterrupt: 